In [7]:
%run ./graph_sampling.ipynb

# Functions to Model Disease Spread

In [8]:
def find_susceptible_neighbors(adj_mat, node_idx, S_nodes):
    nbrhd = get_neighborhood_set_of_node(adj_mat, node_idx)
    
    S_nbrs = []
    for nbr in nbrhd:
        if nbr in S_nodes:
            S_nbrs.append(nbr)
            
    return S_nbrs

# A: weighted adjacency matrix, encoding time of contact between two individuals
# p: probability of the disease spreading from one individual to another, per unit contact time (i.e. per 20 sec interval)
# q: probability of moving from infected to recovered state
def Binomial_SIR_epidemic_model(A, p, q, DEBUG):
    n = A.shape[0]
    adj_mat = np.where(A != 0, 1, 0)
    
    # Init all nodes to Susceptible state
    SIR_states = {}
    S_nodes = range(0, n)
    I_nodes = []
    R_nodes = []
    for i in range(0, n):
        SIR_states[i] = 'S'
    
    # Randomly select a node to move to Infected state
    infected_seed_node = np.random.randint(0, n)
    SIR_states[infected_seed_node] = 'I'
    I_nodes.append(infected_seed_node)
    S_nodes.remove(infected_seed_node)
    
    # Run simulation as long as there are still susceptible nodes which have a nonzero probability of being infected
    # i.e. check if there are still any infected nodes, and check if there are still any susceptible nodes
    # T tracks how long the epidemic lasts, in days
    T = 0
    num_infected = len(I_nodes)
    num_susceptible = len(S_nodes)
    while num_infected > 0 and num_susceptible > 0:
        
        for infected_node in I_nodes:
            # First, simulate disease spreading from infected to susceptible individuals
            S_nbrs = find_susceptible_neighbors(adj_mat, infected_node, S_nodes)
            if not S_nbrs:
                if DEBUG:
                    print "Infected node {} has no susceptible neighbors".format(infected_node)
            else:
                for S_nbr in S_nbrs:
                    # Simulate disease spread for each susceptible neighbor
                    r = np.random.binomial(A[infected_node][S_nbr], p)
                    if r > 0:
                        if DEBUG:
                            print "S node {} was infected on day {}".format(S_nbr, T)
                        I_nodes.append(S_nbr)
                        S_nodes.remove(S_nbr)
                        SIR_states[S_nbr] = 'I'
                        
            # Second, simulate disease recovery/death of a node
            r = np.random.binomial(1, q)
            if r > 0:
                if DEBUG:
                    print "I node {} moved to recovered state on day {}".format(infected_node, T)
                I_nodes.remove(infected_node)
                R_nodes.append(infected_node)
                SIR_states[infected_node] = 'R'
        
        # Increment the time of epidemic
        T += 1
        
        # Recount the number of infected and susceptible nodes
        num_infected = len(I_nodes)
        num_susceptible = len(S_nodes)
        
    return SIR_states
    
        
                        
    

In [9]:
#SIR_results = Binomial_SIR_epidemic_model(agg_mat_LyonSchool, 0.0005, 0.1, True)

In [10]:
#import itertools
#[(k, len(list(v))) for k, v in itertools.groupby(sorted(SIR_results.values()))]

In [11]:
#NS_pres_LyonSchool = graph_sampling_algo_uniform_node_sampling(agg_mat_pres_LyonSchool, 0.4)

In [12]:
#SIR_results = Binomial_SIR_epidemic_model(NS_pres_LyonSchool, 0.0005, 0.1, True)